Part 1

In [1]:
import pandas as pd
import requests
import lxml.html as lh

Scrapting Wiki page:

In [16]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Creading a dataframe in Pandas:

In [64]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Cleaning the dataframe:

In [59]:
df.columns = ['PostCode','Borough','Neighbourhood']
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df = df.replace('\n',' ', regex=True)
df.head()

,Neighbourhood,PostCode,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


In [60]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Neighbourhood,PostCode,Borough
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,Harbourfront,M5A,Downtown Toronto
3,Regent Park,M5A,Downtown Toronto
4,Lawrence Heights,M6A,North York


In [61]:
df = df.groupby(['PostCode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['PostCode','Borough','Neighbourhood']
df.head()

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern ,Rouge ,Malvern ,Rouge ,Malvern..."
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union ,Highla..."
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill ,Guildwood ,..."
3,M1G,Scarborough,"Woburn ,Woburn ,Woburn ,Woburn ,Woburn ,Woburn"
4,M1H,Scarborough,"Cedarbrae ,Cedarbrae ,Cedarbrae ,Cedarbrae ,Ce..."


In [62]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [63]:
df.shape

(103, 3)